In [1]:
import math

# Population size
N = 1974750

# Confidence Level (95%) Z-Score
Z = 2.576

# Margin of error (e.g., 5%)
e = 0.02

# Assuming p = 0.5 (maximum variability)
p = 0.5

# Calculate the sample size
sample_size = ((Z**2 * p * (1 - p)) / e**2) / (1 + ((Z**2 * p * (1 - p)) / (e**2 * N)))
sample_size = math.ceil(sample_size)

sample_size

4139